In [1]:
import numpy as np
from gurobipy import *
from scipy.stats import beta
import matplotlib.pyplot as plt
import random
import time

In [2]:

M = 4
#supply

N = 32
#demand

T = 12
#period

num_i = M
num_j = N
num_t = T

bar_I = np.array([1 for i in range(num_i)])
# I_binary,initial solution for benders

Z = np.array([(250*num_j)/num_i for i in range(num_i)])
# Z'i_capacity

demand = 200+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_t*num_j, 0.01), 2, 4)
zeta=demand.reshape(num_t,num_j)
for t in range(num_t):
    for j in range(num_j):
        zeta[t,j]=round(zeta[t,j],0)
#zeta=np.array([[0 for j in range(num_j)]for t in range(num_t)])
#demand1 = 100+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
#demand2 = 400+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
#demand3 = 100+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
#demand4 = 100+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
#for j in range(num_j):
#    zeta[0,j]=round(demand1[j],2)
#    zeta[1,j]=round(demand2[j],2)
#    zeta[2,j]=round(demand3[j],2)
#    zeta[3,j]=round(demand4[j],2)

zetau=zeta
#U'tj 需求均值

zetaup=1.1*zetau
#zetau'jt 需求上界

zetalow=0.9*zetau
#zetal'jt 需求下界

eta = 50
# eta_price

c =np.array([30 for i in range(num_i)])
# ci_productioncost

C =np.array([10 for j in range(num_j)])
# Cj_inventory cost

#d= np.array([
#[3,6,6,7,7,3,8,8],
#[6,5,5,6,3,3,8,5]])
d= np.array([[
    random.randint(3,8) 
    for j in range(num_j)] 
    for i in range(num_i)])
# dij_transportationcost

F= np.array([20 for i in range(num_i)])
#Fi_fixed construction cost

f=np.array([[0.8
    for j in range(num_j)]
    for t in range(num_t)])
#ftj_target of fill rate


R=16328.445
#R=40390.68
#R=94949
#target profit

num_k = 3
a = [0.3909, 0.01298,0]
b = [0, 0.8701, 1]
#piecewise linear function,k=3

epsilon=0.01


params = dict()
params["num_i"] = num_i
params["num_j"] = num_j
params["num_t"] = num_t
params["num_k"] = num_k
params["Z"] = Z
#params["P"] = P
params["zetau"] = zetau
params["zetaup"] = zetaup
params["zetalow"] = zetalow
params["eta"] = eta
params["c"] = c
params["C"] = C
params["d"] = d
params["F"] = F
params["f"] = f
params["R"] = R
params["a"] = a
params["b"] = b
params["epsilon"] = epsilon
params["bar_I"] = bar_I

In [3]:
def MP(Q, E, iter_i, params):
    num_i=params["num_i"]
    num_j=params["num_j"] 
    num_t=params["num_t"] 
    num_k=params["num_k"]
    Z=params["Z"] 
    zetau=params["zetau"] 
    zetaup=params["zetaup"] 
    zetalow=params["zetalow"]
    eta=params["eta"] 
    c=params["c"]
    C=params["C"]
    d=params["d"] 
    F=params["F"] 
    f=params["f"] 
    R=params["R"]
    a=params["a"] 
    b=params["b"] 
    epsilon=params["epsilon"] 
    bar_I=params["bar_I"]
    
    stop = False
    mp = Model("master_problem")
    
    #decision variables
    I = mp.addVars([i for i in range(num_i)],
                    vtype=GRB.BINARY, name="I")
    beta = mp.addVar(vtype=GRB.CONTINUOUS, name="beta")
    
    #objective function
    mp.setObjective(
        beta,GRB.MINIMIZE)
    
    
    # if sp is unbounded,add feasibility cut
    for item in Q.values():
        mp.addConstr(
            (
            quicksum(item["lamda3"][k,t,n,n,j]*a[k]
                    for k in range(num_k)
                    for t in range(num_t)
                    for n in range(t)
                    for j in range(num_j))
            +quicksum(item["lamda3"][k,t,t,t,j]*a[k]*f[t,j]
                    for k in range(num_k)
                    for t in range(num_t)
                    for j in range(num_j))
            +item["lamda4"]*quicksum(F[i]*I[i]
                            for i in range(num_i))
            +item["theta1"]*R
            +quicksum(item["theta2"][t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(item["theta2"][t,t,t,t,j]*(eta+C[j])
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta3"]*R
            +quicksum(item["theta4"][t,t,j]*eta
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta5"]*R
            +quicksum(item["theta6"][t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(item["theta6"][t,t,t,t,j]*C[j]
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta7"]*R
            -quicksum(item["sigma5"][t,i]*Z[i]*I[i]
                     for t in range(num_t)
                     for i in range(num_i))
            +quicksum(item["sigma8"][t,j]*epsilon
                     for t in range(num_t)
                     for j in range(num_j))
            )<=0)


    # if sp is optimal,add optimality cut
    for item in E.values():
        mp.addConstr(
            (
            quicksum(item["lamda3"][k,t,n,n,j]*a[k]
                    for k in range(num_k)
                    for t in range(num_t)
                    for n in range(t)
                    for j in range(num_j))
            +quicksum(item["lamda3"][k,t,t,t,j]*a[k]*f[t,j]
                    for k in range(num_k)
                    for t in range(num_t)
                    for j in range(num_j))
            +item["lamda4"]*quicksum(F[i]*I[i]
                            for i in range(num_i))
            +item["theta1"]*R
            +quicksum(item["theta2"][t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(item["theta2"][t,t,t,t,j]*(eta+C[j])
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta3"]*R
            +quicksum(item["theta4"][t,t,j]*eta
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta5"]*R
            +quicksum(item["theta6"][t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(item["theta6"][t,t,t,t,j]*C[j]
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta7"]*R
            -quicksum(item["sigma5"][t,i]*Z[i]*I[i]
                     for t in range(num_t)
                     for i in range(num_i))
            +quicksum(item["sigma8"][t,j]*epsilon
                     for t in range(num_t)
                     for j in range(num_j))
            )<=beta)   
        
    mp.optimize()
    
    bar_I = dict()

    if mp.status == GRB.OPTIMAL:
        for i in range(num_i):
            bar_I[i] = 1 if I[i].x > 0.01 else 0
    else:
        print("There is no feasible solution in primal problem")
        stop = True

    return mp, bar_I, stop

In [4]:
def SP(Q, E, bar_I, iter_i, params):
    num_i=params["num_i"]
    num_j=params["num_j"] 
    num_t=params["num_t"] 
    num_k=params["num_k"]
    Z=params["Z"] 
    zetau=params["zetau"] 
    zetaup=params["zetaup"] 
    zetalow=params["zetalow"]
    eta=params["eta"] 
    c=params["c"]
    C=params["C"]
    d=params["d"] 
    F=params["F"] 
    f=params["f"] 
    R=params["R"]
    a=params["a"] 
    b=params["b"] 
    epsilon=params["epsilon"] 
    bar_I=params["bar_I"]
    stop = False
    
    sp = Model("dualsubproblem")
    
    # get unboundedRay
    sp.Params.InfUnbdInfo = 1
    
    #decision variables
    lamda1 = sp.addVars([t for t in range(num_t)],
                        [j for j in range(num_j)],
                        vtype=GRB.CONTINUOUS,name = "lamda1")
    lamda2 = sp.addVars([k for k in range(num_k)],
                        [t for t in range(num_t)],
                        [j for j in range(num_j)],
                        vtype=GRB.CONTINUOUS,name = "lamda2")
    lamda3 = sp.addVars([k for k in range(num_k)],
                        [t for t in range(num_t)],
                        [n for n in range(num_t)],
                        [nn for nn in range(num_t)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "lamda3")
    lamda4 = sp.addVar(lb=-1*GRB.INFINITY,
                       vtype=GRB.CONTINUOUS,name = "lamda4")
       
    theta1 = sp.addVar(vtype=GRB.CONTINUOUS,name = "theta1")
    theta2 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [n for n in range(num_t)],
                        [nn for nn in range(num_t)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "theta2")
    theta3 = sp.addVar(vtype=GRB.CONTINUOUS,name = "theta3")
    theta4 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "theta4")
    theta5 = sp.addVar(vtype=GRB.CONTINUOUS,name = "theta5")
    theta6 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [n for n in range(num_t)],
                        [nn for nn in range(num_t)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "theta6")
    theta7 = sp.addVar(vtype=GRB.CONTINUOUS,name = "theta7")
    theta8 = sp.addVars([tt for tt in range(num_t)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "theta8")
    
    sigma1 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [i for i in range(num_i)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "sigma1")
    sigma2 = sp.addVars([n for n in range(num_t)],
                        [nn for nn in range(num_t)],
                        [i for i in range(num_i)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "sigma2")
    sigma3 = sp.addVars([t for t in range(num_t)],
                        [i for i in range(num_i)],
                        vtype=GRB.CONTINUOUS,name = "sigma3")
    sigma4 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [i for i in range(num_i)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "sigma4")
    sigma5 = sp.addVars([t for t in range(num_t)],
                        [i for i in range(num_i)],
                        vtype=GRB.CONTINUOUS,name = "sigma5")
    sigma6 = sp.addVars([t for t in range(num_t)],
                        [i for i in range(num_i)],
                        [j for j in range(num_j)],
                        vtype=GRB.CONTINUOUS,name = "sigma6")
    sigma7 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [i for i in range(num_i)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "sigma7")
    sigma8 = sp.addVars([t for t in range(num_t)],
                        [j for j in range(num_j)],
                        vtype=GRB.CONTINUOUS,name = "sigma8")
    
    #objective function
    sp.setObjective(
        (
            quicksum(lamda3[k,t,n,n,j]*a[k]
                    for k in range(num_k)
                    for t in range(num_t)
                    for n in range(t)
                    for j in range(num_j))
            +quicksum(lamda3[k,t,t,t,j]*a[k]*f[t,j]
                    for k in range(num_k)
                    for t in range(num_t)
                    for j in range(num_j))
            +lamda4*quicksum(F[i]*bar_I[i]
                            for i in range(num_i))
            +theta1*R
            +quicksum(theta2[t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(theta2[t,t,t,t,j]*(eta+C[j])
                     for t in range(num_t)
                     for j in range(num_j))
            +theta3*R
            +quicksum(theta4[t,t,j]*eta
                     for t in range(num_t)
                     for j in range(num_j))
            +theta5*R
            +quicksum(theta6[t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(theta6[t,t,t,t,j]*C[j]
                     for t in range(num_t)
                     for j in range(num_j))
            +theta7*R
            -quicksum(sigma5[t,i]*Z[i]*bar_I[i]
                     for t in range(num_t)
                     for i in range(num_i))
            +quicksum(sigma8[t,j]*epsilon
                     for t in range(num_t)
                     for j in range(num_j))
    ),GRB.MAXIMIZE)
    
    #constraints for dual value
    #S
    d1=sp.addConstrs(
        (lamda1[t,j]
         -quicksum(lamda2[k,t,j]
                   for k in range(num_k))
        )==0
        for j in range(num_j)
        for t in range(num_t)
    )    
    d2=sp.addConstrs(
        (lamda1[t,j]*zetau[t,j]
         -quicksum(lamda3[k,t,t,t,j]
                   for k in range(num_k))
        )==0
        for j in range(num_j)
        for t in range(num_t)
    )
    #u,v
    d3=sp.addConstrs(
        (-lamda2[k,t,j]*zetaup[nn,j]
         +lamda3[k,t,n,nn,j]
        )<=0
        for k in range(num_k)
        for t in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
    )
    d4=sp.addConstrs(
        (lamda2[k,t,j]*zetalow[nn,j]
         -lamda3[k,t,n,nn,j]
        )<=0
        for k in range(num_k)
        for t in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
    )
    #A,B
    d5=sp.addConstrs(
        (
        -theta1*zetaup[nn,j]
        -theta2[t,tt,n,nn,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
    )
    
    d6=sp.addConstrs(
        (
        theta1*zetalow[nn,j]
        +theta2[t,tt,n,nn,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
    )
    #G,H
    d7=sp.addConstrs(
        (
        -theta3*zetaup[tt,j]
        -theta4[t,tt,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for j in range(num_j)
    )
    d8=sp.addConstrs(
        (
        theta3*zetalow[tt,j]
        +theta4[t,tt,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for j in range(num_j)
    )
    #o,s
    d9=sp.addConstrs(
        (
        -theta5*zetaup[nn,j]
        -theta6[t,tt,n,nn,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
    )    
    d10=sp.addConstrs(
        (
        theta5*zetalow[nn,j]
        +theta6[t,tt,n,nn,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
    )
    #r,w
    d11=sp.addConstrs(
        (
        -theta7*zetaup[tt,j]
        -theta8[tt,j]
        )<=0
        for tt in range(num_t)
        for j in range(num_j)
    )    
    d12=sp.addConstrs(
        (
        theta7*zetalow[tt,j]
        +theta8[tt,j]
        )<=0
        for tt in range(num_t)
        for j in range(num_j)
    )
    #x,z
    d13=sp.addConstrs(
        (
        -sigma3[t,i]*zetaup[tt,j]
        +sigma4[t,tt,i,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
    )    
    d14=sp.addConstrs(
        (
        sigma3[t,i]*zetalow[tt,j]
        -sigma4[t,tt,i,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
    )
    #g,h
    d15=sp.addConstrs(
        (
        -sigma6[t,i,j]*zetaup[tt,j]
        -sigma7[t,tt,i,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
    )    
    d16=sp.addConstrs(
        (
        sigma6[t,i,j]*zetalow[tt,j]
        +sigma7[t,tt,i,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
    )
    #qt
    d17=sp.addConstrs(
            (
            quicksum(lamda2[k,t,j]*a[k]
                     for k in range(num_k))
            -theta1*d[i,j]
            -theta1*C[j]
            -theta3*d[i,j]
            +theta5*(eta-d[i,j])
            -theta5*C[j]
            +theta7*(eta-d[i,j])
            -sigma3[t,i]
            +sigma6[t,i,j]
            )==0
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
    ) 
    #qtt
    for t in range(num_t):
        for tt in range(num_t):
            for i in range(num_i):
                for j in range(num_j):
                    if tt>=t:
                        sp.addConstr(
                            (
                            quicksum(lamda3[k,t,t,tt,j]*a[k]
                                    for k in range(num_k))
                                +theta2[t,tt,t,tt,j]*C[j]
                                +theta2[t,tt,t,tt,j]
                                +theta4[t,tt,j]*d[i,j]
                                +theta6[t,tt,t,tt,j]*C[j]
                                -theta6[t,tt,t,tt,j]*(eta-d[i,j])
                                -theta8[tt,j]*(eta-d[i,j])
                                +sigma1[t,tt,i,j]
                                -sigma4[t,tt,i,j]
                                -sigma7[t,tt,i,j]
                                )==0
                        )
                    if tt<t:
                        sp.addConstr(
                            (
                            quicksum(lamda3[k,t,t,tt,j]*a[k]
                                    for k in range(num_k))
                                +theta2[t,tt,t,tt,j]*C[j]
                                +theta2[t,tt,t,tt,j]
                                +theta4[t,tt,j]*d[i,j]
                                +theta6[t,tt,t,tt,j]*C[j]
                                -theta6[t,tt,t,tt,j]*(eta-d[i,j])
                                -theta8[tt,j]*(eta-d[i,j])
                                -sigma4[t,tt,i,j]
                                -sigma7[t,tt,i,j]
                                )==0
                        )
    #qn
    d19=sp.addConstrs(
            (
            quicksum(lamda2[k,t,j]*a[k]
                    for k in range(num_k))
            -theta1*C[j]
            -theta5*C[j]
            )==0
        for t in range(num_t)
        for n in range(t)
        for i in range(num_i)
        for j in range(num_j)
    )
    #qnn
    for t in range(num_t):
        for tt in range(num_t):
            for n in range(t):
                for nn in range(t):
                    for i in range(num_i):
                        for j in range(num_j):
                            if nn>=n:
                                sp.addConstr(
                                    (
                                    quicksum(lamda3[k,t,n,nn,j]*a[k]
                                            for k in range(num_k))
                                    +theta2[t,tt,n,nn,j]*C[j]
                                    +theta6[t,tt,n,nn,j]*C[j]
                                    +sigma2[n,nn,i,j]
                                    )==0
                            )
                            if nn<n:
                                sp.addConstr(
                                    (
                                    quicksum(lamda3[k,t,n,nn,j]*a[k]
                                            for k in range(num_k))
                                    +theta2[t,tt,n,nn,j]*C[j]
                                    +theta6[t,tt,n,nn,j]*C[j]
                                    )==0
                            )
    
    #P
    d21=sp.addConstrs(
            (
            -lamda4*c[i]
            +sigma3[t,i]
            -sigma5[t,i]
            )==0
        for t in range(num_t)
        for i in range(num_i)
    ) 
    #pi
    d22=sp.addConstr(
            (
            lamda4-theta1-theta3-theta5-theta7
            )==0
    )
    #alpha
    d23=sp.addConstrs(
            (
            quicksum(lamda3[k,t,t,t,j]*b[k]
                    for k in range(num_k))
            +sigma8[t,j]
            )==1
        for t in range(num_t)
        for j in range(num_j)
    )
        
    sp.optimize()
    
    #get dual value
#    constrs_name=sp.getAttr("ConstrName", sp.getConstrs())
#    y=sp.getAttr("Pi", sp.getConstrs())
        
    #print("sub problem's status:"+str(sp.status)
    if sp.status == GRB.UNBOUNDED:
        item = dict()
        item["lamda1"] = {(t,j):lamda1[t,j].unbdRay
                          for t in range(num_t)
                          for j in range(num_j)}
        item["lamda2"] = {(k,t,j):lamda2[k,t,j].unbdRay
                          for k in range(num_k)
                          for t in range(num_t)
                          for j in range(num_j)}
        item["lamda3"] = {(k,t,n,nn,j):lamda3[k,t,n,nn,j].unbdRay
                          for k in range(num_k)
                          for t in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
        item["lamda4"] = lamda4.unbdRay
        
        item["theta1"] = theta1.unbdRay
        item["theta2"] = {(t,tt,n,nn,j):theta2[t,tt,n,nn,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
        item["theta3"] = theta3.unbdRay
        item["theta4"] = {(t,tt,j):theta4[t,tt,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for j in range(num_j)}
        item["theta5"] = theta5.unbdRay
        item["theta6"] = {(t,tt,n,nn,j):theta6[t,tt,n,nn,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
        item["theta7"] = theta7.unbdRay
        item["theta8"] = {(tt,j):theta8[tt,j].unbdRay 
                          for tt in range(num_t)
                          for j in range(num_j)}
        
        item["sigma1"] = {(t,tt,i,j):sigma1[t,tt,i,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
        item["sigma2"] = {(n,nn,i,j):sigma2[n,nn,i,j].unbdRay 
                          for n in range(num_t)
                          for nn in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
        item["sigma3"] = {(t,i):sigma3[t,i].unbdRay 
                          for t in range(num_t)
                          for i in range(num_i)}
        item["sigma4"] = {(t,tt,i,j):sigma4[t,tt,i,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
        item["sigma5"] = {(t,i):sigma5[t,i].unbdRay 
                          for t in range(num_t)
                          for i in range(num_i)}
        item["sigma6"] = {(t,i,j):sigma6[t,i,j].unbdRay 
                          for t in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
        item["sigma7"] = {(t,tt,i,j):sigma7[t,tt,i,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
        item["sigma8"] = {(t,j):sigma8[t,j].unbdRay 
                          for t in range(num_t)
                          for j in range(num_j)}        
        
        Q[iter_i] = item
        print("Add feasible cut")

    elif sp.status == GRB.OPTIMAL:
        item = dict()
        item["lamda1"] = {(t,j):lamda1[t,j].x
                          for t in range(num_t)
                          for j in range(num_j)}
        item["lamda2"] = {(k,t,j):lamda2[k,t,j].x
                          for k in range(num_k)
                          for t in range(num_t)
                          for j in range(num_j)}
        item["lamda3"] = {(k,t,n,nn,j):lamda3[k,t,n,nn,j].x
                          for k in range(num_k)
                          for t in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
        item["lamda4"] = lamda4.x
                
        item["theta1"] = theta1.x
        item["theta2"] = {(t,tt,n,nn,j):theta2[t,tt,n,nn,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
        item["theta3"] = theta3.x
        item["theta4"] = {(t,tt,j):theta4[t,tt,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for j in range(num_j)}
        item["theta5"] = theta5.x
        item["theta6"] = {(t,tt,n,nn,j):theta6[t,tt,n,nn,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
        item["theta7"] = theta7.x
        item["theta8"] = {(tt,j):theta8[tt,j].x
                          for tt in range(num_t)
                          for j in range(num_j)}
        
        item["sigma1"] = {(t,tt,i,j):sigma1[t,tt,i,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
        item["sigma2"] = {(n,nn,i,j):sigma2[n,nn,i,j].x 
                          for n in range(num_t)
                          for nn in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
        item["sigma3"] = {(t,i):sigma3[t,i].x
                          for t in range(num_t)
                          for i in range(num_i)}
        item["sigma4"] = {(t,tt,i,j):sigma4[t,tt,i,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
        item["sigma5"] = {(t,i):sigma5[t,i].x
                          for t in range(num_t)
                          for i in range(num_i)}
        item["sigma6"] = {(t,i,j):sigma6[t,i,j].x 
                          for t in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
        item["sigma7"] = {(t,tt,i,j):sigma7[t,tt,i,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
        item["sigma8"] = {(t,j):sigma8[t,j].x 
                          for t in range(num_t)
                          for j in range(num_j)}
        
        E[iter_i] = item
        print("Add optimal cut")

    else:
        print("Wrong sub-problem status  " + str(sp.status))
        stop = True
        
#   return sp,constrs_name,y,stop
    return sp,stop

In [5]:
start =time.clock()
#中间写上代码块
E_set = dict()
Q_set = dict()

LB = -1 * 1e10
UB = 1e10

for iter_i in range(1000):
    if np.abs(UB - LB) < 0.01:
        print("Optimal")
        break
        
    print("iteration at " + str(iter_i))
    
    sp,stop = SP(Q_set, E_set, bar_I, iter_i, params)        
    print("Q  " + str(len(Q_set.keys())))
    print("E  " + str(len(E_set.keys())))
    if stop:
        print("Wrong slave problem")
        break
    #item = E_set.get(iter_i, False)
    sp_optimal = sp.objVal 
    print("slave dual objective value")
    print(sp_optimal)
    UB = min(UB, sp_optimal)
    # get dual value of contraints
    #print(constrs_name)
    #print(y)
    #主问题输出
    mp, bar_I, stop = MP(Q_set, E_set, iter_i, params)
    print("bar_I")
    print(bar_I)
    print("master objective value")
    print(mp.objVal)
    if stop:
        print("wrong master problem")
        break
        
    LB = mp.objVal
    print("UB " + str(UB))
    print("LB " + str(LB))
    
end = time.clock()
print('Running time: %s Seconds'%(end-start))  

iteration at 0
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 2013745 rows, 1575269 columns and 6169877 nonzeros
Coefficient statistics:
  Matrix range     [1e-02, 2e+02]
  Objective range  [1e-02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 848992 rows and 1323552 columns (presolve time = 5s) ...
Presolve removed 1325632 rows and 1323680 columns
Presolve time: 7.18s
Presolved: 688113 rows, 251589 columns, 2978389 nonzeros

Elapsed ordering time = 6s
Elapsed ordering time = 10s
Elapsed ordering time = 15s
Elapsed ordering time = 20s
Elapsed ordering time = 25s
Ordering time: 27.44s

Barrier statistics:
 Dense cols : 260
 Free vars  : 2112
 AA' NZ     : 2.395e+07
 Factor NZ  : 3.855e+07 (roughly 700 MBytes of memory)
 Factor Ops : 7.999e+09 (roughly 4 seconds per iteration)
 Threads    : 1

             